In [1]:
import jieba
import jieba.analyse
import csv
import pandas as pd
jieba.set_dictionary('dict.idkrsi.txt')
jieba.analyse.set_stop_words("stopword.goatwang.kang.txt")

In [2]:
stopwords=list()
with open('stopword.goatwang.kang.txt', 'r',encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))

In [3]:
type(stopwords)

list

In [4]:
stopwords[0:20]

['\ufeff\ufeff,',
 '',
 '、',
 '。',
 '“',
 '”',
 '《',
 '》',
 '！',
 '，',
 '：',
 '；',
 '？',
 '人民',
 '末##末',
 '啊',
 '阿',
 '哎',
 '哎呀',
 '哎喲']

In [5]:
news=pd.read_csv("bda2019_dataset/news2.csv",encoding="utf-8")
news.head(2)

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
0,1451580107540_N01,news,yahoo股市,重大要聞,2016/1/1 00:06:00,淪落的2015 竟是78年來最難獲利的一年,鉅亨網,鉅亨網編譯郭照青這一年，獲利真是難上難。<BR>根據CNBC分析報導，興業銀行的數據顯示，2...,https://tw.stock.yahoo.com/news_content/url/d/...
1,1451582807314_N01,news,yahoo股市,重大要聞,2016/1/1 00:52:00,美股盤中─今年最後交易日 道瓊大跌逾150點 注意油價走勢,鉅亨網,鉅亨網編譯郭照青今日為2015年最後交易日，美國股市中場走低，投資人注意油價走勢。<BR>道...,https://tw.stock.yahoo.com/news_content/url/d/...


In [6]:
content=""
count=0
for row in news.index:
    try:
        content=content+news.loc[row,'content']
    except:
        continue
    count+=1
    if count>10000:
        break

In [7]:
templist=jieba.cut(str(content).replace('\n','').replace(',',' '), cut_all=False)
type(templist)

generator

In [8]:
seglist=list(templist)
seglist[0:20]

Building prefix dict from /home/barry/projects/stock/dict.idkrsi.txt ...
Loading model from cache /tmp/jieba.ufc1c7fa02eb193cee2bb78b34d63c017.cache
Loading model cost 0.421 seconds.
Prefix dict has been built succesfully.


['鉅亨網',
 '編譯',
 '郭照',
 '青',
 '這一年',
 '，',
 '獲利',
 '真是',
 '難上',
 '難',
 '。',
 '<',
 'BR',
 '>',
 '根據',
 'CNBC',
 '分析',
 '報導',
 '，',
 '興業']

In [9]:
#newseglist=list()
#for word in seglist:
#    if word not in stopwords:
#        newseglist.append(word)

newseglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist))    # 切詞跟stopwords比對
newseglist[0:20]

['鉅亨網',
 '編譯',
 '郭照',
 '青',
 '這一年',
 '獲利',
 '真是',
 '難上',
 '難',
 'CNBC',
 '分析',
 '報導',
 '興業',
 '銀行',
 '數據',
 '顯示',
 '2015',
 '表現',
 '最佳',
 '資產']

In [10]:
wordcount = {}      # dictionary
for word in newseglist:
    try:
        wordcount[word] = wordcount[word] + 1
    except:
        wordcount[word] = 1

In [11]:
list(wordcount.items())[0:20]   # list( dictionary.items() ) 會轉成list

[('鉅亨網', 892),
 ('編譯', 395),
 ('郭照', 394),
 ('青', 391),
 ('這一年', 14),
 ('獲利', 2582),
 ('真是', 18),
 ('難上', 1),
 ('難', 205),
 ('CNBC', 213),
 ('分析', 981),
 ('報導', 5471),
 ('興業', 36),
 ('銀行', 2748),
 ('數據', 1530),
 ('顯示', 2516),
 ('2015', 3355),
 ('表現', 4133),
 ('最佳', 265),
 ('資產', 893)]

In [49]:
sorted_x = sorted(wordcount.items(), key=lambda kv: kv[1])  # sorted( dictionary.items() ) 也會變成list   # kv[1] 用第2個column排序
sorted_x.reverse()          # ascending 轉 descending
sorted_x[0:20]
sorted_x[:][:20]

[('點', 12932),
 ('市場', 11105),
 ('元', 10255),
 ('指數', 10209),
 ('億元', 10180),
 ('台股', 9118),
 ('美元', 8252),
 ('表示', 7762),
 ('去年', 7646),
 ('今年', 7577),
 ('美國', 6459),
 ('公司', 6166),
 ('中國', 5855),
 ('2016', 5778),
 ('營收', 5551),
 ('報導', 5471),
 ('成長', 5293),
 ('經濟', 5192),
 ('股市', 5125),
 ('外資', 5084)]

In [50]:
len(sorted_x)

91876

In [51]:
#for word, number in sorted_x:
#    judgeword=''.join([str(word), str(number)])

for word in sorted_x:
    judgeword=''.join(map(str, word))                  # str('點'),str(12932)       # 把'點', '12932' 合併成 '點12932'    
    if judgeword.isdigit()==True:                      # 如果全數字       
        sorted_x.remove(word)                          # 從list刪掉
sorted_x[0:20]

[('點', 12932),
 ('市場', 11105),
 ('元', 10255),
 ('指數', 10209),
 ('億元', 10180),
 ('台股', 9118),
 ('美元', 8252),
 ('表示', 7762),
 ('去年', 7646),
 ('今年', 7577),
 ('美國', 6459),
 ('公司', 6166),
 ('中國', 5855),
 ('營收', 5551),
 ('報導', 5471),
 ('成長', 5293),
 ('經濟', 5192),
 ('股市', 5125),
 ('外資', 5084),
 ('持續', 5040)]

In [52]:
len(sorted_x)

87432

In [53]:
pd.DataFrame(dict(sorted_x),index=['number']).head()

,點,市場,元,指數,億元,台股,美元,表示,去年,今年,...,17502.16,101.71,Davitt,維特,月給,流年不利,通殺,McDonald,年成,難上
number,12932,11105,10255,10209,10180,9118,8252,7762,7646,7577,...,1,1,1,1,1,1,1,1,1,1


In [54]:
pd.DataFrame(dict(sorted_x),index=['number']).T.head()

,number
點,12932
市場,11105
元,10255
指數,10209
億元,10180
